In [94]:
#Imports
import requests
import pandas as pd
import pandasql as psql

In [95]:
#Url da Fake Store API
url = 'https://fakestoreapi.com/'

In [96]:
#Definindo a função que faz a chamada à API FakeStore, retornando o json com os resultados ou uma exceção.
def fetch_data(endpoint):
    """
    Busca dados na FakeStore API com base em um endpoint.

    Parâmetros:
    endpoint (str): O Endpoint em que queremos buscar os dados.

    Retorno:
    dict ou None: O JSON de resposta da api se a requisição for bem sucedida, se não for, retorna um None.
    """
    try: 
        response = requests.get(f'{url}{endpoint}')
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados do endpoint {endpoint}: {e}")
        return None

In [97]:
#No desenvolvimento do código notei que várias colunas vieram aninhadas "nested columns" e optei 
#por criar uma função que faça o tratamento do problema.
def normalize_nested_columns(df, coluna):
    """
    Normaliza uma coluna aninhada em um DataFrame.

    Parâmetros:
    df (pd.DataFrame): O DataFrame contendo a coluna aninhada.
    coluna (str): O nome da coluna aninhada à ser normalizada.

    Retorno:
    pd.DataFrame: O DataFrame normalizado.
    """
    
    df_normalizado = pd.json_normalize(df[coluna])
    return df_normalizado

#### dProducts

In [98]:
#Recuperando os dados da dimensão "products"
df_products = pd.DataFrame(fetch_data('products'))

In [99]:
#Tratando a coluna aninhada 'rating'
df_products = df_products.drop(columns='rating') \
                         .join(normalize_nested_columns(df_products,'rating'))

In [100]:
#Separando somente as colunas que serão necessárias para análise
colunas = ['id','category']
df_products = df_products[colunas]

#### dUsers

In [101]:
#Recuperando os dados da dimensão "users"
df_users = pd.DataFrame(fetch_data('users'))

In [102]:
# Tratando as colunas aninhadas 'name' e 'address'
df_users = df_users.drop(columns=['name','address']) \
                   .join(normalize_nested_columns(df_users,'name')) \
                   .join(normalize_nested_columns(df_users,'address'))

In [103]:
#Separando as colunas que serão necessárias para análise
cols = ['id','username','firstname','lastname']
df_users = df_users[cols]

#### fCart

In [104]:
#Recuperando os dados da fato Cart
df_cart = pd.DataFrame(fetch_data('carts'))

In [105]:
#O DataFrame df_cart possui uma coluna com listas de objetos, portanto é necessário "explodir" este objeto em várias linhas.
#É necessário redefinir o index para que o join ocorra corretamente na etapa de normalização.
df_cart_exploded = df_cart.explode('products').reset_index(drop=True)

In [107]:
#Tratando a coluna aninhada "products" 
df_cart = df_cart_exploded.drop(columns=['products']).join(normalize_nested_columns(df_cart_exploded,'products'))

In [109]:
#Para criar o DataFrame final que responde as perguntas de negócio do Case, optei por usar SQL. A biblioteca pandasql permite 
#trabalhar com SQL na manipulação de DataFrames utilizando as funções do sqlite, apesar de não apresentar a performance ideal 
#para bases muito grandes, neste caso será o suficiente.

query = """
    WITH user_products AS (
        SELECT
            u.id as user_id,
            u.firstname || ' ' || u.lastname as user_name,
            MAX(c.date) as date_last_addition,
            p.category,
            SUM(c.quantity) as total_products
        FROM df_cart c
            INNER JOIN df_users u ON c.userId = u.id 
            LEFT JOIN df_products p ON c.productId = p.id
        GROUP BY 
            u.id,
            p.category
        ORDER BY
            u.id ASC
    )
    
    SELECT
        user_id,
        user_name,
        date_last_addition,
        category,
        MAX(total_products) as total_products
    FROM 
        user_products
    GROUP BY 
        user_id
"""
df_analise = psql.sqldf(query, locals())

In [110]:
df_analise

,user_id,user_name,date_last_addition,category,total_products
0,1,john doe,2020-03-02T00:00:00.000Z,men's clothing,25
1,2,david morrison,2020-03-01T00:00:00.000Z,men's clothing,2
2,3,kevin ryan,2020-01-01T00:00:00.000Z,men's clothing,4
3,4,don romer,2020-03-01T00:00:00.000Z,electronics,5
4,8,william hopkins,2020-03-01T00:00:00.000Z,women's clothing,1


In [111]:
#Exportando o df_analise em um arquivo JSON para entrega
df_analise.to_csv('../data/analise.csv', index=False)